<a href="https://colab.research.google.com/github/qinwenshuo/CVAE-linguistic-transfer/blob/main/1_stanza_conllu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import io
import shutil
from tqdm import tqdm
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# as we have more languages, change the "language_list" below
language_list = ['English']
language = input(f"Please choose one language from the list: {language_list}\nEnter your choice: ")
nativeness = input("Native or Learner?\nEnter your choice: ")
input_directory = os.path.join('/content/drive/MyDrive/cvae_project/1_extract_txt_format/', language, nativeness)
subdirectories = os.listdir(input_directory)
subdirectories = [i.replace('.zip', '') for i in subdirectories if i != 'statistics']
corpus = input(f"Please choose one corpus from the list: {subdirectories}\nEnter your choice: ")
input_directory = os.path.join(input_directory, corpus+'.zip')
output_directory = f'/content/{corpus}/'
archived_output_directory = f'/content/drive/MyDrive/cvae_project/2_conllu_format/{language}/{nativeness}/'
os.makedirs(archived_output_directory, exist_ok=True)
archived_output_directory += corpus

print("Input directory:", input_directory)
print("Output directory:", output_directory)
print(f"Archived output directory: {archived_output_directory}")


Please choose one language from the list: ['English']
Enter your choice: English
Native or Learner?
Enter your choice: Learner
Please choose one corpus from the list: ['ArabCC', 'ICLE', 'WriCLE', 'ICNALE', 'TOEFL', 'PELIC', 'WriCLE_informal', 'BAWE', 'Gachon', 'CLC']
Enter your choice: CLC
Input directory: /content/drive/MyDrive/cvae_project/1_extract_txt_format/English/Learner/CLC.zip
Output directory: /content/CLC/
Archived output directory: /content/drive/MyDrive/cvae_project/2_conllu_format/English/Learner/CLC


In [ ]:
!pip install stanza
import stanza

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.5/802.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 42.2 MB/s eta 0:00:00


In [ ]:
### Getting POS and STEM features from Stanza ###

def get_features(sentence, lg, l1, parse_info):

    parse_results = []
    for sent in parse_info.sentences:
        for i in range(len(sent.words)):
            word = sent.words[i]
            w_id = word.id
            w_text = word.text
            w_lemma = word.lemma
            w_upos = word.upos
            w_xpos = word.xpos
            w_feats = word.feats
            w_head = word.head
            w_deprel = word.deprel
            parse_results.append([w_id, w_text, w_lemma, w_upos, w_xpos, w_feats, w_head, w_deprel, lg, l1])
        parse_results.append([])

    return parse_results

In [ ]:
lg_code = {'Arabic': 'ar', 'English': 'en', 'Spanish': 'es', 'Croatian': 'hr', 'Czech': 'cs',
           'German': 'de', 'Italian': 'it', 'Norwegian': 'no', 'Portuguese': 'pt', 'Latvian': 'lv',
           'Icelandic': 'is', 'Finnish': 'fi', 'Chinese': 'zh', 'Korean': 'ko'}
lg_dict = {'ar': 'ar_padt', 'en': 'en_ewt', 'es': 'es_ancora', 'hr': 'hr_set',
           'cs': 'cs_pdt', 'de': 'de_gsd', 'it':'it_partut', 'no': 'no_bokmaal',
           'pt': 'pt_gsd', 'lv': 'lv_lvtb', 'is': 'is_modern', 'fi': 'fi_tdt',
           'zh': 'zh_gsdsimp', 'ko': 'ko_kaist'}

lg = lg_code[language]
nlp = stanza.Pipeline(lang=lg, processors='tokenize,pos,lemma,depparse')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


In [ ]:
# Call the function to clear the /content/ directory after using the unzipped files
def clear_content_directory():
    content_dir = '/content/'
    exempt_folders = ['drive', 'sample_data']

    for item in os.listdir(content_dir):
        item_path = os.path.join(content_dir, item)
        if os.path.isdir(item_path) and item not in exempt_folders:
            for root, dirs, files in os.walk(item_path, topdown=False):
                for file in files:
                    file_path = os.path.join(root, file)
                    os.remove(file_path)
                for dir in dirs:
                    dir_path = os.path.join(root, dir)
                    os.rmdir(dir_path)
            os.rmdir(item_path)
        elif os.path.isfile(item_path):
            os.remove(item_path)

In [ ]:
def parse_native_corpus():
    os.makedirs(output_directory, exist_ok=True)
    for filename in tqdm(os.listdir(input_extraction_path)):
        if filename.endswith('.txt'):
            file_path = os.path.join(input_extraction_path, filename)
            name = filename.replace('.txt', '')
            if name + '.conllu' not in os.listdir(output_directory) or os.stat(output_directory + name + '.conllu').st_size == 0:
                with open(file_path, 'r') as file:
                    file_content = file.read()
                parse_info = nlp(file_content)
                features = get_features(file_content, lg, lg, parse_info)
                formatted_output = ""
                for row in features:
                    if not row:
                        formatted_output += "\n"
                    else:
                        formatted_output += "\t".join(map(str, row)) + "\n"
                with open(os.path.join(output_directory, name+'.conllu'), 'w') as out_file:
                    out_file.write(formatted_output.strip())


def parse_learner_corpus():
    os.makedirs(output_directory, exist_ok=True)
    print(f'All languages: {os.listdir(input_extraction_path)}')
    for language in os.listdir(input_extraction_path):
        language_path = os.path.join(input_extraction_path, language)
        for filename in tqdm(os.listdir(language_path), desc = f'Processing {language}'):
            file_path = os.path.join(language_path, filename)
            name = filename.replace('.txt', '')
            if name + '.conllu' not in os.listdir(output_directory) or os.stat(output_directory + name + '.conllu').st_size == 0:
                with open(file_path, 'r') as file:
                    file_content = file.read()
                parse_info = nlp(file_content)
                features = get_features(file_content, lg, language, parse_info)
                formatted_output = ""
                for row in features:
                    if not row:
                        formatted_output += "\n"
                    else:
                        formatted_output += "\t".join(map(str, row)) + "\n"
                with open(os.path.join(output_directory, name+'.conllu'), 'w') as out_file:
                    out_file.write(formatted_output.strip())

In [ ]:
# clear temp files before unzipping
clear_content_directory()

In [ ]:
# unzip input files
input_extraction_path = f'/content/unzipped_files/{corpus}'
shutil.unpack_archive(input_directory, input_extraction_path, 'zip')
# unzip output files so that you can continue parsing
if os.path.exists(archived_output_directory+'.zip'):
    output_extraction_path = f'/content/{corpus}'
    shutil.unpack_archive(archived_output_directory+'.zip', output_extraction_path, 'zip')

if nativeness == 'Native':
    parse_native_corpus()
elif nativeness == 'Learner':
    parse_learner_corpus()

All languages: ['Dutch', 'French', 'Russian', 'Portuguese', 'Korean', 'Spanish', 'German', 'Chinese', 'Polish', 'Japanese', 'Catalan', 'Turkish', 'Italian', 'Thai', 'Swedish', 'Greek']


Processing Greek: 100%|██████████| 148/148 [00:20<00:00,  7.20it/s]


In [ ]:
# archive output files to speed up transmission through API
if os.path.exists(archived_output_directory+'.zip'):
    os.remove(archived_output_directory+'.zip')
shutil.make_archive(archived_output_directory, 'zip', output_directory)


'/content/drive/MyDrive/cvae_project/2_conllu_format/English/Learner/CLC.zip'